Debemos instalar la libreria catboost dado que no viene por defecto.

In [ ]:
pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.cluster import KMeans

# **Carga de datos para el fit**

In [ ]:
train = pd.read_csv('data/train.csv')
data = train.copy()


# **Preprocesado del DataFrame**
Usamos imputación con la estrategia adecuada para cada columna

# **Imputación para columnas numéricas**




Esta función transforma valores categóricos relacionados con los rangos de matrícula universitaria a valores numéricos promedio.



In [ ]:
def convertir_valor_matricula(valor):
    if pd.isna(valor):
        return np.nan
    elif 'Menos de 500 mil' in valor:
        return 250000  # Asignamos un valor promedio entre 0 y 500 mil
    elif 'Entre 500 mil y menos de 1 millón' in valor:
        return 750000  # Promedio entre 500 mil y 1 millón
    elif 'Entre 1 millón y menos de 2.5 millones' in valor:
        return 1750000  # Promedio entre 1 y 2.5 millones
    elif 'Entre 2.5 millones y menos de 4 millones' in valor:
        return 3250000  # Promedio entre 2.5 y 4 millones
    elif 'Entre 4 millones y menos de 5.5 millones' in valor:
        return 4750000  # Promedio entre 4 y 5.5 millones
    elif 'Entre 5.5 millones y menos de 7 millones' in valor:
        return 6250000  # Promedio entre 5.5 y 7 millones
    elif 'Más de 7 millones' in valor:
        return 7500000  # Asignamos un valor mínimo representativo superior a 7 millones
    elif 'No pagó matrícula' in valor:
        return 0  # Asumimos que no se pagó nada
    else:
        return np.nan  # Para cualquier caso que no coincida

Esta función transforma valores categóricos que representan rangos de horas trabajadas semanalmente en valores numéricos promedio o representativos.

In [ ]:
def convertir_horas_trabajadas(valor):
    if isinstance(valor, str):
        if "Entre" in valor:
            partes = valor.split('y')
            min_val = float(partes[0].split(' ')[-2])  # Obtener el penúltimo elemento
            max_val = float(partes[1].split(' ')[-2])  # Obtener el penúltimo elemento de la segunda parte
            return (min_val + max_val) / 2  # Retornar el promedio del rango

        elif "Más de" in valor:
            return float(valor.split(' ')[2])  # Convertir a número

        elif "Menos de" in valor:
            return float(valor.split(' ')[2])  # Convertir a número

        elif "0" in valor:
            return 0  # Devolver 0 en número

    return np.nan  # Devolver NaN si no es un valor válido

## **Aplicación de funciones e imputaciones para columnas numéricas**

In [ ]:
# Aplicar las funciones de conversión
data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)

data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)

# Imputar valores faltantes
num_imputer = SimpleImputer(strategy='mean')

data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']] = num_imputer.fit_transform(
    data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']]
)


# Agregar num_columns
num_columns = data.select_dtypes(include=['number']).columns
num_columns = [col for col in num_columns if col not in ['ID']]

# **Imputación para columnas categóricas**
Esta celda utiliza la técnica de imputación para manejar valores faltantes en columnas categóricas. Específicamente, aplica el método de la moda para rellenar los valores faltantes en las columnas indicadas.

In [ ]:
cat_columns = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_ESTRATOVIVIENDA',
               'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']

cat_imputer = SimpleImputer(strategy='most_frequent')  # Usamos la moda para imputar valores categóricos


data[cat_columns] = cat_imputer.fit_transform(data[cat_columns])


# **2. Conversión de variables categóricas**
Esta celda transforma columnas categóricas binarias en un formato numérico (1 y 0). Específicamente, mapea los valores 'Si' y 'No' en las columnas indicadas por la lista binary_columns.

In [ ]:
binary_columns = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']


def map_binary_columns(data, binary_columns, mapping={'Si': 1, 'No': 0}):
    for col in binary_columns:
        data[col] = data[col].map(mapping)
    return data

data = map_binary_columns(data, binary_columns)

**Codificación de variables categóricas multiclase con diccionario**

Esta celda transforma los valores categóricos de la columna FAMI_ESTRATOVIVIENDA, que representan estratos socioeconómicos en formato de texto (por ejemplo, 'Estrato 1'), en valores numéricos. Adicionalmente, se maneja el valor 'Sin Estrato' y los valores faltantes (NaN) asignándoles un valor de 0.


In [ ]:
# Crear un diccionario para asignar el valor numérico a cada valor en letra de la columna FAMI_ESTRATOVIVIENDA
moda_estrato = data['FAMI_ESTRATOVIVIENDA'].mode()[0]

estrato_dict = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0,
    np.nan: moda_estrato
}

data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)


<ipython-input-248-0cb37154a7a4>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)


Esta celda transforma los valores categóricos en la columna RENDIMIENTO_GLOBAL, que representan niveles de rendimiento (por ejemplo, 'bajo', 'alto'), en valores numéricos mediante un mapeo definido en un diccionario.

In [ ]:
rendimiento_mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
data['RENDIMIENTO_GLOBAL'] = data['RENDIMIENTO_GLOBAL'].map(rendimiento_mapping)

Esta celda aplica codificación de etiquetas (Label Encoding) a las columnas categóricas especificadas en la lista label_columns. Este proceso convierte los valores categóricos en números enteros secuenciales. Adicionalmente, almacena los codificadores (LabelEncoder) utilizados, permitiendo reutilizarlos posteriormente para transformar datos nuevos o invertir el proceso.



In [ ]:
label_columns = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']

def apply_label_encoding(data, columns, label_encoders={}):
    for col in columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le
    return data, label_encoders

data, label_encoders = apply_label_encoding(data, label_columns)

# **3. One-Hot Encoding para variables indicadoras (educación de padres y madres)**

Esta celda implementa una función para aplicar codificación One-Hot Encoding a columnas categóricas especificadas. One-Hot Encoding convierte cada categoría en una columna binaria, donde un valor de 1 indica la presencia de la categoría y un valor de 0 su ausencia. La función también maneja valores faltantes imputándolos con un valor predeterminado.

In [ ]:

columns_to_encode = ['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE']

def apply_onehot_encoding(data, columns, fill_value='No Aplica'):
  onehot_encoders = {}

  for col in columns:
        data[col].fillna(fill_value, inplace=True)
        unique_values = sorted(data[col].unique())
        onehot_encoders[col] = unique_values
        onehot_data = pd.get_dummies(data[col], prefix=col)
        data = pd.concat([data, onehot_data], axis=1)
        data.drop(col, axis=1, inplace=True)
  return data, onehot_encoders

data, onehot_encoders = apply_onehot_encoding(data, columns_to_encode)

<ipython-input-251-780daa6885c1>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(fill_value, inplace=True)
<ipython-input-251-780daa6885c1>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

# **4. Escalado de variables numéricas**

Esta celda normaliza las columnas numéricas y otras seleccionadas del DataFrame utilizando el escalador MinMaxScaler de scikit-learn. La normalización transforma los valores de las columnas a un rango específico (en este caso, de -1 a 1).

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
data[num_columns] = scaler.fit_transform(data[num_columns])

data['ESTU_PRGM_ACADEMICO'] = scaler.fit_transform(data[['ESTU_PRGM_ACADEMICO']])
data['ESTU_PRGM_DEPARTAMENTO'] = scaler.fit_transform(data[['ESTU_PRGM_DEPARTAMENTO']])
data['FAMI_ESTRATOVIVIENDA'] = scaler.fit_transform(data[['FAMI_ESTRATOVIVIENDA']])


# ***Guardamos el preprocesamiento en un diccionario para usarlo en el conjunto de prueba***



---



In [ ]:
preprocessing_objects = {
    'num_imputer': num_imputer,
    'cat_imputer': cat_imputer,
    'label_encoders': label_encoders,
    'onehot_encoders': onehot_encoders,
}

# **Dataset preprocesado**

In [ ]:
data

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,...,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,Cluster
0,904256,0.933333,-0.366420,-0.733333,0.666667,-0.333333,0.000000,1,0,2,...,False,False,False,False,False,False,False,False,True,0
1,645256,0.933333,-0.474129,-0.800000,-0.133333,-1.000000,0.000000,0,0,0,...,False,False,False,False,False,False,False,True,False,1
2,308367,0.333333,0.729673,-0.733333,-0.133333,1.000000,0.000000,1,0,0,...,False,False,False,False,False,True,False,False,False,0
3,470353,-0.200000,-0.970433,0.733333,0.266667,-1.000000,0.333333,1,0,3,...,False,True,False,False,False,False,False,False,False,0
4,989032,0.933333,0.915523,-0.933333,-0.133333,0.700000,0.000000,1,0,1,...,False,False,False,True,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,-0.200000,-0.695882,0.133333,-0.800000,0.033333,-0.333333,1,1,2,...,False,False,False,False,False,True,False,False,False,4
692496,754213,0.933333,0.915523,0.400000,-0.133333,1.000000,0.000000,1,0,0,...,False,False,False,False,True,False,False,False,False,0
692497,504185,-1.000000,-0.801478,-0.733333,-0.533333,-0.333333,0.000000,1,1,1,...,False,False,False,False,False,True,False,False,False,0
692498,986620,-0.200000,0.915523,0.866667,-0.133333,-0.333333,-0.666667,0,1,0,...,False,False,False,True,False,False,False,False,False,5


# **Entrenamiento**

Este bloque de código tiene como objetivo realizar un análisis de agrupamiento utilizando K-Means para dividir a los estudiantes en diferentes clústeres según las características de estrato y departamento. Posteriormente, se entrena un modelo de clasificación independiente para cada clúster utilizando CatBoostClassifier con el propósito de predecir el rendimiento académico (RENDIMIENTO_GLOBAL) de los estudiantes.

In [ ]:

# Selección de características para clustering
clustering_features = data[['FAMI_ESTRATOVIVIENDA', 'ESTU_PRGM_DEPARTAMENTO']]

# Aplicar K-Means para agrupar estudiantes
n_clusters = 6  # Número de clústeres, puedes ajustar según los datos
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
data['Cluster'] = kmeans.fit_predict(clustering_features)

# Inicializar diccionario para almacenar los modelos por clúster
models_by_cluster = {}

for cluster in range(n_clusters):
    print(f"--- Cluster {cluster} ---")

    # Filtrar datos del clúster
    cluster_data = data[data['Cluster'] == cluster]

    # Separar características y etiqueta
    X_train = cluster_data.drop(columns=['RENDIMIENTO_GLOBAL', 'ID', 'Cluster'])
    y_train = cluster_data['RENDIMIENTO_GLOBAL']

    # Crear y entrenar el modelo CatBoostClassifier
    model = CatBoostClassifier(
        iterations=200,  # Número de iteraciones o árboles en el modelo
        depth=10,  # Profundidad máxima de los árboles
        learning_rate=0.1,  # Tasa de aprendizaje
        l2_leaf_reg=8,  # Regularización L2 en las hojas
        colsample_bylevel=0.8,  # Proporción de características a usar en cada nivel del árbol
        min_data_in_leaf=20,  # Número mínimo de datos en cada hoja
        loss_function='MultiClass',  # Función de pérdida para clasificación multiclase
        random_seed=42,  # Semilla aleatoria para asegurar reproducibilidad
        bagging_temperature=1.0,  # Control de aleatoriedad en el muestreo
        verbose=0  # No mostrar el progreso del entrenamiento
    )

    model.fit(X_train, y_train)

    # Evaluar precisión en el conjunto de entrenamiento
    y_train_pred = model.predict(X_train)
    accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Accuracy para Cluster {cluster}: {accuracy:.4f}")

    # Guardar el modelo entrenado
    models_by_cluster[cluster] = model

--- Cluster 0 ---
Accuracy para Cluster 0: 0.4876
--- Cluster 1 ---
Accuracy para Cluster 1: 0.4903
--- Cluster 2 ---
Accuracy para Cluster 2: 0.5251
--- Cluster 3 ---
Accuracy para Cluster 3: 0.5359
--- Cluster 4 ---
Accuracy para Cluster 4: 0.5397
--- Cluster 5 ---
Accuracy para Cluster 5: 0.4541


# **Carga de datos para el Predict**

In [ ]:
test = pd.read_csv('data/test.csv')
test_data = test.copy()

# **Preprocesamiento del conjunto de prueba**
Esta celda aplica varias etapas de preprocesamiento al conjunto de prueba (test_data) para asegurar que esté en el mismo formato que los datos utilizados para entrenar los modelos. El objetivo es garantizar la consistencia entre las características del conjunto de prueba y el modelo.

In [ ]:
# Eliminar columna innecesaria 'Unnamed: 0' del conjunto de datos de prueba
test_data = test_data.drop('Unnamed: 0', axis=1)
test_data  # Muestra el conjunto de datos después de eliminar la columna

# Aplicar la función de conversión para el valor de matrícula universitaria
test_data['ESTU_VALORMATRICULAUNIVERSIDAD'] = test_data['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)

# Aplicar la función de conversión para las horas trabajadas por semana
test_data['ESTU_HORASSEMANATRABAJA'] = test_data['ESTU_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)

# Imputar valores faltantes en las columnas 'ESTU_VALORMATRICULAUNIVERSIDAD' y 'ESTU_HORASSEMANATRABAJA' con el imputador numérico
test_data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']] = num_imputer.fit_transform(
    test_data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']]
)

# Selección de columnas numéricas para la normalización
num_columns = test_data.select_dtypes(include=['number']).columns  # Filtrar solo las columnas numéricas
num_columns = [col for col in num_columns if col not in ['ID']]  # Excluir la columna 'ID'

# Aplicar MinMaxScaler para escalar las características numéricas en el rango [-1, 1]
test_data[num_columns] = scaler.fit_transform(test_data[num_columns])

# Definir las columnas categóricas para la imputación
cat_columns = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_ESTRATOVIVIENDA',
               'FAMI_TIENEINTERNET','ESTU_PAGOMATRICULAPROPIO']

# Imputar los valores faltantes en las columnas categóricas utilizando el imputador para categorías
test_data[cat_columns] = cat_imputer.fit_transform(test_data[cat_columns])

# Reemplazar los valores de estrato en la columna 'FAMI_ESTRATOVIVIENDA' con el diccionario de estratos
test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)

# Mapear las columnas binarias usando la función definida previamente
binary_columns = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']
test_data = map_binary_columns(test_data, binary_columns)

# Aplicar codificación one-hot a las columnas 'FAMI_EDUCACIONMADRE' y 'FAMI_EDUCACIONPADRE'
columns_to_encode = ['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE']
test_data, onehot_encoders = apply_onehot_encoding(test_data, columns_to_encode)

# Aplicar codificación de etiquetas a las columnas 'ESTU_PRGM_ACADEMICO' y 'ESTU_PRGM_DEPARTAMENTO'
label_columns = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']
test_data, label_encoders = apply_label_encoding(test_data, label_columns)

# Normalizar las columnas 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO' y 'FAMI_ESTRATOVIVIENDA' utilizando MinMaxScaler
test_data['ESTU_PRGM_ACADEMICO'] = scaler.fit_transform(test_data[['ESTU_PRGM_ACADEMICO']])
test_data['ESTU_PRGM_DEPARTAMENTO'] = scaler.fit_transform(test_data[['ESTU_PRGM_DEPARTAMENTO']])
test_data['FAMI_ESTRATOVIVIENDA'] = scaler.fit_transform(test_data[['FAMI_ESTRATOVIVIENDA']])


<ipython-input-294-e85810f91ab3>:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['FAMI_ESTRATOVIVIENDA'] = test_data['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)
<ipython-input-251-780daa6885c1>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(fill_value, inplace=True)


In [ ]:
test_data

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONMADRE_Educación profesional completa,...,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta
0,550236,-1.000000,0.991285,-0.666667,-0.933333,-0.333333,0.000000,1,1,False,...,False,False,False,False,False,False,False,False,True,False
1,98545,0.333333,-0.971678,-0.933333,-0.133333,0.700000,-0.333333,1,0,False,...,False,False,False,False,False,False,True,False,False,False
2,499179,0.933333,-0.021786,-0.733333,-0.533333,-1.000000,0.000000,1,0,False,...,False,False,False,False,False,False,False,True,False,False
3,782980,-0.200000,-0.503268,0.800000,-0.533333,0.700000,-0.666667,0,0,False,...,False,False,False,False,False,True,False,False,False,False
4,785185,0.933333,-0.969499,-0.800000,-0.133333,0.033333,-0.333333,1,0,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,-0.200000,-0.969499,-0.733333,-0.133333,1.000000,-0.666667,1,1,False,...,False,False,False,False,False,True,False,False,False,False
296782,209415,-1.000000,-0.461874,0.266667,-0.533333,-1.000000,0.333333,1,0,True,...,False,False,False,False,False,False,False,False,False,False
296783,239074,0.933333,-0.461874,-0.733333,-0.133333,1.000000,0.000000,1,0,True,...,False,False,False,False,False,False,True,False,False,False
296784,963852,-0.200000,-0.211329,-0.933333,0.666667,0.033333,0.000000,1,0,True,...,False,False,False,False,False,False,False,False,False,False


# **Predicción**
Esta celda utiliza los modelos entrenados para realizar predicciones del rendimiento global de estudiantes en un conjunto de prueba (test_data). Además, maneja casos en los que no hay un modelo entrenado para un departamento específico, asignando predicciones por defecto. Finalmente, los resultados se guardan en un archivo CSV para ser envaluados en Kaggle.

In [ ]:
results = []

# Asignar clústeres al conjunto de prueba utilizando el modelo K-Means entrenado
test_clustering_features = test_data[['FAMI_ESTRATOVIVIENDA','ESTU_PRGM_DEPARTAMENTO']]
test_data['Cluster'] = kmeans.predict(test_clustering_features)

# Predicción para cada clúster
for cluster, model in models_by_cluster.items():
    print(f"Realizando predicciones para el Cluster: {cluster}")

    # Filtrar los datos del test para el clúster
    test_cluster_data = test_data[test_data['Cluster'] == cluster]

    if test_cluster_data.empty:
        continue

    # Separar las características del conjunto de prueba
    X_test = test_cluster_data.drop(columns=['ID', 'Cluster'], errors='ignore')

    # Realizar predicción
    y_test_pred = model.predict(X_test)

    # Convertir las predicciones a etiquetas de rendimiento
    y_test_pred = y_test_pred.ravel()
    y_test_pred_labels = [list(rendimiento_mapping.keys())[int(pred)] for pred in y_test_pred]

    # Guardar los resultados
    results.extend(zip(test_cluster_data['ID'], y_test_pred_labels))

# Crear archivo de envío
submission = pd.DataFrame(results, columns=['ID', 'RENDIMIENTO_GLOBAL'])
submission.to_csv('submission[Cluster].csv', index=False)
print("Archivo de predicciones creado: 'submission[Clusterl].csv'")


Realizando predicciones para el Cluster: 0
Realizando predicciones para el Cluster: 1
Realizando predicciones para el Cluster: 2
Realizando predicciones para el Cluster: 3
Realizando predicciones para el Cluster: 4
Realizando predicciones para el Cluster: 5
Archivo de predicciones creado: 'submission[Clusterl].csv'


Esta celda carga el archivo de predicciones generado (submission-IA.csv) y analiza la distribución de las etiquetas de rendimiento global predichas

In [ ]:
sub = pd.read_csv('submission[Cluster].csv')
sub['RENDIMIENTO_GLOBAL'].value_counts()

,count
RENDIMIENTO_GLOBAL,
bajo,89116
alto,87656
medio-bajo,60386
medio-alto,59628
